In [1]:
import pandas as pd
import numpy as np

In [2]:
import pandas.util.testing as tm

tm.N = 3


def unpivot(frame):
    N, K = frame.shape
    data = {'value': frame.to_numpy().ravel('F'),
            'variable': np.asarray(frame.columns).repeat(N),
            'date': np.tile(np.asarray(frame.index), K)}
    return pd.DataFrame(data, columns=['date', 'variable', 'value'])


df = unpivot(tm.makeTimeDataFrame())

In [3]:
df.head()

,date,variable,value
0,2000-01-03,A,0.062076
1,2000-01-04,A,0.434799
2,2000-01-05,A,0.613620
3,2000-01-03,B,-0.069989
4,2000-01-04,B,-0.028231


In [4]:
# Making a pivot table
df.pivot(index='date', columns='variable', values='value')

variable,A,B,C,D
date,,,,
2000-01-03,0.062076,-0.069989,0.308451,-0.463208
2000-01-04,0.434799,-0.028231,-0.940056,-0.754656
2000-01-05,0.613620,-0.114217,0.085772,0.584895


In [5]:
# Omitting values arg when there is more than one value column
df['value2'] = df['value'] * 2
pivoted = df.pivot(index='date', columns='variable')
pivoted

value                                  value2            \
variable           A         B         C         D         A         B   
date                                                                     
2000-01-03  0.062076 -0.069989  0.308451 -0.463208  0.124152 -0.139979   
2000-01-04  0.434799 -0.028231 -0.940056 -0.754656  0.869598 -0.056462   
2000-01-05  0.613620 -0.114217  0.085772  0.584895  1.227240 -0.228435   

                                
variable           C         D  
date                            
2000-01-03  0.616901 -0.926416  
2000-01-04 -1.880111 -1.509313  
2000-01-05  0.171545  1.169790

In [6]:
pivoted['value']

variable,A,B,C,D
date,,,,
2000-01-03,0.062076,-0.069989,0.308451,-0.463208
2000-01-04,0.434799,-0.028231,-0.940056,-0.754656
2000-01-05,0.613620,-0.114217,0.085772,0.584895


In [7]:
# Reshaping by stacking transforms column labels to a index level
print(pivoted.index)
print(pivoted.stack().index)

DatetimeIndex(['2000-01-03', '2000-01-04', '2000-01-05'], dtype='datetime64[ns]', name='date', freq=None)
MultiIndex([('2000-01-03', 'A'),
            ('2000-01-03', 'B'),
            ('2000-01-03', 'C'),
            ('2000-01-03', 'D'),
            ('2000-01-04', 'A'),
            ('2000-01-04', 'B'),
            ('2000-01-04', 'C'),
            ('2000-01-04', 'D'),
            ('2000-01-05', 'A'),
            ('2000-01-05', 'B'),
            ('2000-01-05', 'C'),
            ('2000-01-05', 'D')],
           names=['date', 'variable'])


In [8]:
stacked = pivoted.stack()
stacked

value    value2
date       variable                    
2000-01-03 A         0.062076  0.124152
           B        -0.069989 -0.139979
           C         0.308451  0.616901
           D        -0.463208 -0.926416
2000-01-04 A         0.434799  0.869598
           B        -0.028231 -0.056462
           C        -0.940056 -1.880111
           D        -0.754656 -1.509313
2000-01-05 A         0.613620  1.227240
           B        -0.114217 -0.228435
           C         0.085772  0.171545
           D         0.584895  1.169790

In [9]:
# unstack does the opposite
stacked.unstack()

value                                  value2            \
variable           A         B         C         D         A         B   
date                                                                     
2000-01-03  0.062076 -0.069989  0.308451 -0.463208  0.124152 -0.139979   
2000-01-04  0.434799 -0.028231 -0.940056 -0.754656  0.869598 -0.056462   
2000-01-05  0.613620 -0.114217  0.085772  0.584895  1.227240 -0.228435   

                                
variable           C         D  
date                            
2000-01-03  0.616901 -0.926416  
2000-01-04 -1.880111 -1.509313  
2000-01-05  0.171545  1.169790

In [10]:
# If there is a multiIndex you can indicate the level to stack or unstack
pivoted.stack(0)

variable                  A         B         C         D
date                                                     
2000-01-03 value   0.062076 -0.069989  0.308451 -0.463208
           value2  0.124152 -0.139979  0.616901 -0.926416
2000-01-04 value   0.434799 -0.028231 -0.940056 -0.754656
           value2  0.869598 -0.056462 -1.880111 -1.509313
2000-01-05 value   0.613620 -0.114217  0.085772  0.584895
           value2  1.227240 -0.228435  0.171545  1.169790

In [11]:
stacked.unstack(0) # Could also be the name (if named)

value                           value2                      
date     2000-01-03 2000-01-04 2000-01-05 2000-01-03 2000-01-04 2000-01-05
variable                                                                  
A          0.062076   0.434799   0.613620   0.124152   0.869598   1.227240
B         -0.069989  -0.028231  -0.114217  -0.139979  -0.056462  -0.228435
C          0.308451  -0.940056   0.085772   0.616901  -1.880111   0.171545
D         -0.463208  -0.754656   0.584895  -0.926416  -1.509313   1.169790

In [12]:
columns = pd.MultiIndex.from_tuples([('A', 'cat'), ('B', 'dog'),
                                         ('B', 'cat'), ('A', 'dog')],
                                        names=['exp', 'animal'])
    
index = pd.MultiIndex.from_product([('bar', 'baz', 'foo', 'qux'),
                                        ('one', 'two')],
                                       names=['first', 'second']) 

df2 = pd.DataFrame(np.random.randn(8, 4), index=index, columns=columns)

df3 = df2.iloc[[0, 1, 4, 7], [1, 2]]
df3

exp                  B          
animal             dog       cat
first second                    
bar   one    -0.924796  0.442826
      two    -0.606846 -0.965465
foo   one     1.987150  0.017493
qux   two     0.264450  1.260337

In [13]:
# Setting a missing value that results from unstacking
df3.unstack(fill_value=-1e9)

exp                B                                          
animal           dog                         cat              
second           one           two           one           two
first                                                         
bar    -9.247956e-01 -6.068458e-01  4.428259e-01 -9.654654e-01
foo     1.987150e+00 -1.000000e+09  1.749269e-02 -1.000000e+09
qux    -1.000000e+09  2.644502e-01 -1.000000e+09  1.260337e+00

In [14]:
# Melting a data frame (contracts it)
cheese = pd.DataFrame({'first': ['John', 'Mary'],
                           'last': ['Doe', 'Bo'],
                           'height': [5.5, 6.0],
                           'weight': [130, 150]})
cheese

,first,last,height,weight
0,John,Doe,5.5,130
1,Mary,Bo,6.0,150


In [15]:
cheese.melt(['first','last'])

,first,last,variable,value
0,John,Doe,height,5.5
1,Mary,Bo,height,6.0
2,John,Doe,weight,130.0
3,Mary,Bo,weight,150.0


In [16]:
cheese.melt(['first','last'], var_name='quantity')

,first,last,quantity,value
0,John,Doe,height,5.5
1,Mary,Bo,height,6.0
2,John,Doe,weight,130.0
3,Mary,Bo,weight,150.0


In [17]:
# Using wide_to_long()
dft = pd.DataFrame({"A1970": {0: "a", 1: "b", 2: "c"},
                        "A1980": {0: "d", 1: "e", 2: "f"},
                        "B1970": {0: 2.5, 1: 1.2, 2: .7},
                        "B1980": {0: 3.2, 1: 1.3, 2: .1},
                        "X": dict(zip(range(3), np.random.randn(3)))
                       })
    

dft["id"] = dft.index
dft

,A1970,A1980,B1970,B1980,X,id
0,a,d,2.5,3.2,1.582810,0
1,b,e,1.2,1.3,-0.423885,1
2,c,f,0.7,0.1,-0.846462,2


In [18]:
pd.wide_to_long(dft, ["A","B"], i='id', j='year')

,,X,A,B
id,year,,,
0,1970,1.582810,a,2.5
1,1970,-0.423885,b,1.2
2,1970,-0.846462,c,0.7
0,1980,1.582810,d,3.2
1,1980,-0.423885,e,1.3
2,1980,-0.846462,f,0.1


In [19]:
# Calculating statistics
df2

exp                  A         B                   A
animal             cat       dog       cat       dog
first second                                        
bar   one     0.281850 -0.924796  0.442826 -0.456157
      two     0.770621 -0.606846 -0.965465 -0.263401
baz   one     0.670799  0.188422 -0.278930  0.284275
      two    -0.290738 -0.461530 -2.583590 -0.153554
foo   one     0.469236  1.987150  0.017493  0.662038
      two     0.054296 -0.058599 -1.187593 -0.303721
qux   one    -1.195725 -0.809969  1.025139 -0.006008
      two    -0.735039  0.264450  1.260337 -0.751639

In [20]:
df2.stack().mean(1)

first  second  animal
bar    one     cat       0.362338
               dog      -0.690476
       two     cat      -0.097422
               dog      -0.435123
baz    one     cat       0.195934
               dog       0.236349
       two     cat      -1.437164
               dog      -0.307542
foo    one     cat       0.243364
               dog       1.324594
       two     cat      -0.566648
               dog      -0.181160
qux    one     cat      -0.085293
               dog      -0.407989
       two     cat       0.262649
               dog      -0.243595
dtype: float64

In [21]:
df2.stack().mean(1).unstack()

animal             cat       dog
first second                    
bar   one     0.362338 -0.690476
      two    -0.097422 -0.435123
baz   one     0.195934  0.236349
      two    -1.437164 -0.307542
foo   one     0.243364  1.324594
      two    -0.566648 -0.181160
qux   one    -0.085293 -0.407989
      two     0.262649 -0.243595

In [22]:
# Another way
df2.groupby(level='animal', axis=1).mean()

animal             cat       dog
first second                    
bar   one     0.362338 -0.690476
      two    -0.097422 -0.435123
baz   one     0.195934  0.236349
      two    -1.437164 -0.307542
foo   one     0.243364  1.324594
      two    -0.566648 -0.181160
qux   one    -0.085293 -0.407989
      two     0.262649 -0.243595

In [23]:
# Pivoting with aggregations
import datetime

df = pd.DataFrame({'A': ['one', 'one', 'two', 'three'] * 6,
                       'B': ['A', 'B', 'C'] * 8,
                       'C': ['foo', 'foo', 'foo', 'bar', 'bar', 'bar'] * 4,
                       'D': np.random.randn(24),
                       'E': np.random.randn(24),
                       'F': [datetime.datetime(2013, i, 1) for i in range(1, 13)]
                       + [datetime.datetime(2013, i, 15) for i in range(1, 13)]})
    

df.head(15)

,A,B,C,D,E,F
0,one,A,foo,-0.093170,-0.063371,2013-01-01
1,one,B,foo,1.186655,-0.167304,2013-02-01
2,two,C,foo,0.582788,0.098410,2013-03-01
3,three,A,bar,1.807518,0.199765,2013-04-01
4,one,B,bar,-1.295422,0.441347,2013-05-01
5,one,C,bar,-1.132937,0.543509,2013-06-01
6,two,A,foo,-0.207871,0.914541,2013-07-01
7,three,B,foo,1.024642,0.676372,2013-08-01
8,one,C,foo,0.239898,-0.630096,2013-09-01
9,one,A,bar,0.777608,0.828611,2013-10-01


In [24]:
pd.pivot_table(df.head(15), values='D',index=['B'], columns=['A','C'], aggfunc=np.sum)

A       one               three                 two          
C       bar       foo       bar       foo       bar       foo
B                                                            
A  0.777608 -0.040398  1.807518       NaN       NaN -0.207871
B -1.295422  1.031757       NaN  1.024642  0.443069       NaN
C -1.132937  0.239898 -2.023236       NaN       NaN -0.144277

In [25]:
pd.pivot_table(df.head(15), values='D',index=['B'], columns=['A','C'])

A       one               three                 two          
C       bar       foo       bar       foo       bar       foo
B                                                            
A  0.777608 -0.020199  1.807518       NaN       NaN -0.207871
B -1.295422  0.515879       NaN  1.024642  0.443069       NaN
C -1.132937  0.239898 -2.023236       NaN       NaN -0.072139

In [26]:
# Getting a total row and column for whatever aggregation is called
df.pivot_table(index=['A', 'B'], columns='C', margins=True, aggfunc=np.std)

D                             E                    
C             bar       foo       All       bar       foo       All
A     B                                                            
one   A  0.077087  0.103197  0.497672  0.415404  0.322172  0.371295
      B  1.636861  0.948620  1.155672  0.000766  1.075374  1.005356
      C  1.260457  0.231630  0.762317  0.252881  0.387392  0.494525
three A  0.296366       NaN  0.296366  0.797819       NaN  0.797819
      B       NaN  0.384875  0.384875       NaN  1.782536  1.782536
      C  1.403332       NaN  1.403332  1.264650       NaN  1.264650
two   A       NaN  0.103504  0.103504       NaN  1.057365  1.057365
      B  0.680510       NaN  0.680510  1.675131       NaN  1.675131
      C       NaN  0.926206  0.926206       NaN  1.962698  1.962698
All      1.294896  0.565050  0.963484  1.005838  1.366154  1.172045

In [27]:
# Cross tablulations
pd.crosstab(df.A, df.B)

B,A,B,C
A,,,
one,4,4,4
three,2,2,2
two,2,2,2


In [28]:
# Normalizing to show percentages
pd.crosstab(df.A, df.B, normalize=True)

B,A,B,C
A,,,
one,0.166667,0.166667,0.166667
three,0.083333,0.083333,0.083333
two,0.083333,0.083333,0.083333


In [29]:
# Percentages across columns
pd.crosstab(df.A, df.B, normalize='columns')

B,A,B,C
A,,,
one,0.50,0.50,0.50
three,0.25,0.25,0.25
two,0.25,0.25,0.25


In [30]:
# Percentages across rows
pd.crosstab(df.A, df.B, normalize='index')

B,A,B,C
A,,,
one,0.333333,0.333333,0.333333
three,0.333333,0.333333,0.333333
two,0.333333,0.333333,0.333333


In [31]:
# Cross tab with more than one index or column
pd.crosstab(df.A, [df.B, df.C])

B       A       B       C    
C     bar foo bar foo bar foo
A                            
one     2   2   2   2   2   2
three   2   0   0   2   2   0
two     0   2   2   0   0   2

In [32]:
# Adding margins (or totals)
pd.crosstab(df.A, df.B, margins=True)

B,A,B,C,All
A,,,,
one,4,4,4,12
three,2,2,2,6
two,2,2,2,6
All,8,8,8,24


In [33]:
# Aggregating values
pd.crosstab(df.A, df.B, values = df.E, aggfunc=np.sum, margins=True)

B,A,B,C,All
A,,,,
one,1.398629,-0.971639,0.017055,0.444045
three,1.527815,-1.168142,1.310340,1.670014
two,3.324423,-2.576315,-2.578854,-1.830745
All,6.250867,-4.716095,-1.251459,0.283313


In [34]:
# Creating groupings or intervals of equal width
ages = np.array([10, 15, 13, 12, 23, 25, 28, 59, 60])
pd.cut(ages, bins=3)

[(9.95, 26.667], (9.95, 26.667], (9.95, 26.667], (9.95, 26.667], (9.95, 26.667], (9.95, 26.667], (26.667, 43.333], (43.333, 60.0], (43.333, 60.0]]
Categories (3, interval[float64]): [(9.95, 26.667] < (26.667, 43.333] < (43.333, 60.0]]

In [35]:
# Predefined bins
pd.cut(ages, bins=[0,18,35,70])

[(0, 18], (0, 18], (0, 18], (0, 18], (18, 35], (18, 35], (18, 35], (35, 70], (35, 70]]
Categories (3, interval[int64]): [(0, 18] < (18, 35] < (35, 70]]

In [36]:
# Dummy variables
df = pd.DataFrame({'key': list('bbacab'), 'data1': range(6)})
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [37]:
pd.get_dummies(df.key)

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [38]:
# Set a column prefix
dummies = pd.get_dummies(df.key, prefix='key')
dummies

,key_a,key_b,key_c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [39]:
# Join to the original dataset
df.join(dummies)

,key,data1,key_a,key_b,key_c
0,b,0,0,1,0
1,b,1,0,1,0
2,a,2,1,0,0
3,c,3,0,0,1
4,a,4,1,0,0
5,b,5,0,1,0


In [40]:
# Using the dataframe all non-object or category columns are included
pd.get_dummies(df)

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


In [41]:
# Alternatively, indicate which columns to get dummy vars for
pd.get_dummies(df, columns=['key'])

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


In [42]:
# Combining with cut
values = np.random.randn(10)
bins = [0,0.2,0.4,0.6,0.8,1]

pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,0,0,0
1,0,0,0,0,0
2,0,0,0,1,0
3,0,0,0,0,0
4,0,1,0,0,0
5,0,0,0,0,0
6,0,0,0,1,0
7,0,0,0,0,0
8,0,1,0,0,0
9,0,0,0,0,0


In [43]:
# Obtaining k-1 variabels requires drop_first
pd.get_dummies(df.key, drop_first=True)

,b,c
0,1,0
1,1,0
2,0,0
3,0,1
4,0,0
5,1,0


In [44]:
np.random.seed([3, 1415])
n = 20
cols = np.array(['key', 'row', 'item', 'col'])
df = cols + pd.DataFrame((np.random.randint(5, size=(n, 4))// [2, 1, 2, 1]).astype(str))
df.columns = cols
df = df.join(pd.DataFrame(np.random.rand(n, 2).round(2)).add_prefix('val'))
df

,key,row,item,col,val0,val1
0,key0,row3,item1,col3,0.81,0.04
1,key1,row2,item1,col2,0.44,0.07
2,key1,row0,item1,col0,0.77,0.01
3,key0,row4,item0,col2,0.15,0.59
4,key1,row0,item2,col1,0.81,0.64
5,key1,row2,item2,col4,0.13,0.88
6,key2,row4,item1,col3,0.88,0.39
7,key1,row4,item1,col1,0.10,0.07
8,key1,row0,item2,col4,0.65,0.02
9,key1,row2,item0,col2,0.35,0.61


In [45]:
# Pivot table with multiple aggregations
pd.pivot_table(data = df, values='val0', index='row', columns = 'col', aggfunc=['mean','sum'])

mean                              sum                        
col   col0   col1   col2   col3  col4  col0  col1  col2  col3  col4
row                                                                
row0  0.77  0.605    NaN  0.860  0.65  0.77  1.21   NaN  0.86  0.65
row2  0.13    NaN  0.395  0.500  0.25  0.13   NaN  0.79  0.50  0.50
row3   NaN  0.310    NaN  0.545   NaN   NaN  0.31   NaN  1.09   NaN
row4   NaN  0.100  0.395  0.760  0.24   NaN  0.10  0.79  1.52  0.24

In [46]:
# Exploding a list-like column
keys = ['panda1', 'panda2', 'panda3']
values = [['eats', 'shoots'], ['shoots', 'leaves'], ['eats', 'leaves']]
df = pd.DataFrame({'keys': keys, 'values': values})
df

,keys,values
0,panda1,"[eats, shoots]"
1,panda2,"[shoots, leaves]"
2,panda3,"[eats, leaves]"


In [47]:
df.explode('values')

,keys,values
0,panda1,eats
0,panda1,shoots
1,panda2,shoots
1,panda2,leaves
2,panda3,eats
2,panda3,leaves


In [48]:
pd.__version__

'0.25.0'